In [15]:
import cv2
import csv
import pandas as pd 
import re
from PIL import Image
from PIL import ImageEnhance
from PIL import ImageStat
import math
import numpy as np
from shutil import copyfile

In [16]:
def adjust_brightness(input_image, output_image, factor):
    image = Image.open(input_image)
    enhancer_object = ImageEnhance.Brightness(image)
    out = enhancer_object.enhance(factor)
    out.save(output_image)

def brightness(im_file):
   im = Image.open(im_file)
   stat = ImageStat.Stat(im)
   r,g,b = stat.rms
   return math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2))    


In [17]:
FilePath = "/home/nramvinojen/Programs/Workbench/24Jan2019/"
#RunFolder = "Default"
LoGLimit = 5 

In [22]:
#extract details from DataSetInfo.csv
DataSetInfoCol1 = pd.read_csv(FilePath + "Dataset/DataSetInfo.csv", usecols=[1])
DataSetLabels = DataSetInfoCol1.values #Column 1 = labels
DataSetLabels = [item for sublist in DataSetLabels for item in sublist] #making it into list
NumDataSet = len(DataSetLabels) #number of dataset

DataSetInfoCol3 = pd.read_csv(FilePath + "Dataset/DataSetInfo.csv", usecols=[3])
DataSetTRorTest = DataSetInfoCol3.values #Column 1 = TR or Test
DataSetTRorTest = [item for sublist in DataSetTRorTest for item in sublist]#making it into list

#extract details from ClassNameInfo.csv
ClassNameInfoCol0 = pd.read_csv(FilePath + "Dataset/ClassNameInfo.csv", usecols=[0])
ClassNames = ClassNameInfoCol0.values #Column 0 = names
ClassNames = [item for sublist in ClassNames for item in sublist] #making it into list
NumClass = len(ClassNames) #number of class

#variable to count number of dataset for each label
NumDataSetPerLabel =[[0 for _ in range(NumClass)] for _ in range(2)] #initialising the count, [ [#TR0, #TR1, ....], [#Test, #Test1, ....]]

In [23]:
for SetIter in range(NumDataSet):  
    FolderName = "DS" + str(SetIter)
    print("--------------------------------------------------------------------------------------------DS" + str(SetIter) )
    
    #StartTime of video extracted from info.csv
    Info = open(FilePath + "Dataset/DatasetVid/" + FolderName + '/info.csv',"r")
    InfoData = list(csv.reader(Info))
    Info.close()

    #Fixation data extract from fixation.csv
    FixationCol1 = pd.read_csv(FilePath + "Dataset/DatasetVid/" + FolderName + "/fixations.csv", usecols=[1])
    FixationCol1Data = FixationCol1.values #Column 1 = Starttime

    #extracting basedata from fixation.csv
    #Column13 = All the start time of frames in a fixation
    FixationCol13 = pd.read_csv(FilePath + "Dataset/DatasetVid/" + FolderName + "/fixations.csv", usecols=[13])
    BaseDataStartTime = FixationCol13.values # 
    FixationPosX = pd.read_csv(FilePath + "Dataset/DatasetVid/" + FolderName + "/fixations.csv", usecols=[5])
    FixationPosXData = FixationPosX.values

    #Extracting norm_pos_y from fixations.csv
    FixationPosY = pd.read_csv(FilePath + "Dataset/DatasetVid/" + FolderName + "/fixations.csv", usecols=[6])
    FixationPosYdata = FixationPosY.values
    
    VidStartTime = float(InfoData[5][1]) #Start time of the video from info.csv

    #List conversion
    FixationStartTime = [item for sublist in FixationCol1Data for item in sublist] #start_timestamp
    CoordinateX = [item for sublist in FixationPosXData for item in sublist] #Coordinate X for all fixations
    CoordinateY = [item for sublist in FixationPosYdata for item in sublist] #Coordinate Y for all fixations
    
    NumFixation  = len(FixationStartTime) #Total number of fixations 
    for i in range(NumFixation): #Syncing Fixation time and Video time
        FixationStartTime[i] -= VidStartTime
        
    #Setting a reference frame from the video in 1000ms
    vidcap = cv2.VideoCapture(FilePath + "Dataset/DatasetVid/" + FolderName +'/world.mp4')
    vidcap.set(cv2.CAP_PROP_POS_MSEC,1000) #Extracting frame at 1000ms
    success,RefImage = vidcap.read()         
    FrameXResolution = RefImage.shape[1] #Extracting X resolution for reference image
    FrameYResolution = RefImage.shape[0] #Extracting Y resolution for reference image
    
    vidcap.set(cv2.CAP_PROP_POS_AVI_RATIO,1)
    vidcap_len = vidcap.get(cv2.CAP_PROP_POS_MSEC) #getting the total len of the video
    
    LabelSplit = list(DataSetLabels[SetIter]) # splits the string into individual letters "C0" -> 'C', '0'
    if DataSetTRorTest[SetIter] == "TR" :
        IndexTRorTest = 0
    else : 
        IndexTRorTest = 1
    
    #Laplacian of Gaussian for extracting best frame
    for i in range(NumFixation):
        tmpstr = BaseDataStartTime[i][0] ### second 0 for string only
        FixationBaseDataStartTime = re.findall(r'[\d\.\d]+', tmpstr)
        FixationBaseDataStartTime = [float(i) for i in FixationBaseDataStartTime] #Extracted base data start time for single fixtion  

        for m in range(len(FixationBaseDataStartTime)):
          FixationBaseDataStartTime[m] -= VidStartTime
        maxi = 0 #Stores the maximum value of LoG
        ind_i = 0 #Stores the index of maximum value
        
        Limit = LoGLimit
        if(len(FixationBaseDataStartTime)<LoGLimit):
            Limit = len(FixationBaseDataStartTime)
        for j in range(Limit):  
            if FixationBaseDataStartTime[j]*1000 < vidcap_len-100 :
                vidcap.set(cv2.CAP_PROP_POS_MSEC,1000*FixationBaseDataStartTime[j])      # video cueing
                success,image = vidcap.read()
            else :
                #print("debug1")
                vidcap.set(cv2.CAP_PROP_POS_MSEC,vidcap_len-100)
                success,image = vidcap.read()
            
            fm = cv2.Laplacian(image, cv2.CV_64F).var() #Calculating LoG
            #Storing maximum value and index
            if fm>maxi:
                maxi=fm
                ind_i=j
        
        if FixationBaseDataStartTime[ind_i]*1000 < vidcap_len-100 :
            vidcap.set(cv2.CAP_PROP_POS_MSEC,1000*FixationBaseDataStartTime[ind_i])      # video cueing
            success,frame = vidcap.read()
        else :
            #print("debug2")
            vidcap.set(cv2.CAP_PROP_POS_MSEC,vidcap_len-100)
            success,frameframe = vidcap.read()
            
        CorrectedCoordinateX = CoordinateX[i] * FrameXResolution 
        CorrectedCoordinateY = (1-CoordinateY[i]) * FrameYResolution 

        Xlower = int(CorrectedCoordinateX)-130
        Xhigher = int(CorrectedCoordinateX)+130
        Ylower = int(CorrectedCoordinateY)-110
        Yhigher = int(CorrectedCoordinateY)+110
        if Xlower<0:
            Xlower = 0
        if Xhigher>FrameXResolution:
            Xhigher = FrameXResolution
        if Ylower<0:
            Ylower = 0
        if Yhigher>FrameYResolution:
            Yhigher = FrameYResolution 

        CroppedFrame = frame[Ylower:Yhigher,Xlower:Xhigher]         
        
        if i != NumFixation-1:
            ImageName = str(DataSetLabels[SetIter]) + "_" + str(DataSetTRorTest[SetIter]) + "_" + str(NumDataSetPerLabel[IndexTRorTest][int(LabelSplit[1])]) + "_F_" + str(i)
        else:
            ImageName = str(DataSetLabels[SetIter]) + "_" +str(DataSetTRorTest[SetIter]) + "_"  + str(NumDataSetPerLabel[IndexTRorTest][int(LabelSplit[1])]) + "_T_" + str(NumDataSetPerLabel[IndexTRorTest][int(LabelSplit[1])])
            
        cv2.imwrite(FilePath + "Runs/"+ RunFolder +"/Frames/" + ImageName +".jpg", CroppedFrame)       
                
        #Adjusting image brightness (Contrast,Gamma,Sharpening : Future Scope , based on accuracy)
        RefBrightness = 160.61508828512333
        FrameBrightness = brightness(FilePath + "Runs/"+ RunFolder +"/Frames/" + ImageName +".jpg")
        BrightnessFactor = RefBrightness/FrameBrightness
        adjust_brightness(FilePath + "Runs/"+ RunFolder +"/Frames/" + ImageName +".jpg",FilePath + "Runs/"+ RunFolder + "/Frames/" + ImageName +".jpg",(BrightnessFactor))
        print (ImageName)

    #copying the fixations.cvs and renaming
    FixationFileName = str(DataSetLabels[SetIter]) + "_" + str(DataSetTRorTest[SetIter]) + "_" + str(NumDataSetPerLabel[IndexTRorTest][int(LabelSplit[1])]) + "_fixations.csv"
    copyfile(FilePath + "Dataset/DatasetVid/" + FolderName + '/fixations.csv', FilePath +  "Runs/"+ RunFolder + "/Fixations/" +  FixationFileName)
    
    #increment the count for individual labels
    NumDataSetPerLabel[IndexTRorTest][int(LabelSplit[1])] = NumDataSetPerLabel[IndexTRorTest][int(LabelSplit[1])] +1 # increment the correct array position
    

    
print("All fixations and Target frames are saved.")
print("All fixations.csv are copied and renamed.")


--------------------------------------------------------------------------------------------DS0
C4_TR_0_F_0
C4_TR_0_F_1
C4_TR_0_F_2
C4_TR_0_F_3
C4_TR_0_F_4
C4_TR_0_F_5
C4_TR_0_F_6
C4_TR_0_F_7
C4_TR_0_F_8
C4_TR_0_F_9
C4_TR_0_F_10
C4_TR_0_F_11
C4_TR_0_F_12
C4_TR_0_F_13
C4_TR_0_F_14
C4_TR_0_F_15
C4_TR_0_T_0
--------------------------------------------------------------------------------------------DS1
C6_TR_0_F_0
C6_TR_0_F_1
C6_TR_0_F_2
C6_TR_0_F_3
C6_TR_0_F_4
C6_TR_0_F_5
arumai
arumai part 2
C6_TR_0_T_0
--------------------------------------------------------------------------------------------DS2
C5_TR_0_F_0
C5_TR_0_F_1
C5_TR_0_F_2
C5_TR_0_T_0
--------------------------------------------------------------------------------------------DS3
C0_TR_0_F_0
C0_TR_0_F_1
C0_TR_0_F_2
C0_TR_0_T_0
--------------------------------------------------------------------------------------------DS4
C3_TR_0_F_0
C3_TR_0_F_1
C3_TR_0_F_2
C3_TR_0_F_3
C3_TR_0_F_4
C3_TR_0_F_5
C3_TR_0_F_6
C3_TR_0_T_0
-------------

In [24]:
#creating Class.csv, and placing inside DataSetImage folder

#clearing the existing file
filename = FilePath + "Runs/"+ RunFolder + "/ClassInfo.csv"
f = open(filename, "w+") # opening the file with w+ mode truncates the file
f.close()

#writing the first row, Title
WriteList =  [str(NumClass) , "", ""]
with open(FilePath + "Runs/"+ RunFolder + "/ClassInfo.csv", 'a', newline='') as graphFile:     
    graphFileWriter = csv.writer(graphFile, delimiter = ',')
    graphFileWriter.writerow(WriteList)

# writing the remaining rows. number of rows = number of class
for i in range(NumClass) :
    WriteList =  [ str(ClassNames[i])  ,  str(NumDataSetPerLabel[0][i]) ,  str(NumDataSetPerLabel[1][i]) ]
    with open(FilePath + "Runs/"+ RunFolder + "/ClassInfo.csv", 'a', newline='') as graphFile:      
        graphFileWriter = csv.writer(graphFile, delimiter = ',')
        graphFileWriter.writerow(WriteList)
        
print("ClassInfo.csv is created")

ClassInfo.csv is created
